# Spark构建分类模型
分类通常是指将事物分成不同的类别。在分类模型中，我们期望根据一组特征来判断类别，这些特征代表来物体、事件或上下文相关的属性（变量）。
##  Spark常见的分类模型
- ***逻辑回归*** : 根据现有数据对分类边界线建立回归公式，以此进行分类。 
- *** SVM *** : 找到一个分割平面尽可能远的支撑向量，用这个平面来分类向量。
- ***决策树*** : 决策树算法是一种从跟节点自上而下的方法，中每一个步骤中通过评估特征分裂的信息增益，最后选出分割数据集最优的特征
- ***朴素贝叶斯*** : 朴素贝叶斯是一个概率模型，通过计算给定数据点在某个类别的概率来进行预测。（朴素贝叶斯模型假定每个特征分配到某个类别的概率是独立分布的）

## 本章节分为以下几个部分
* 一、从数据中抽取合适的特征
* 二、训练分类模型
* 三、使用分类模型
* 四、评估模型的性能
* 五、改进模型性能以及参数调优

##   一、从数据中抽取合适的特征
从Kaggle/StumbleUpon evergreen分类数据集中抽取特征  
数据源自Kaggle比赛，连接http://www.kaggle.com/c/stumbleupon/data    
读取第一行数据

In [1]:
val PATH = "file:///Users/lzz/work/SparkML/"
val rawData = sc.textFile( PATH + "data/kaggle/train_noheader.tsv")
val records = rawData.map(line => line.split("\t"))
records.first()

Array("http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html", "4042", "{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees ...

可以看到前面四列分别是 url，页面ID，文本内容，页面的类别,后面的22列包含各种恶扬的数值或者类属特征。最后一列为目标值，－1为长久，0为短暂。  
缓存数据方便后面重复使用并且统计数据样本的数目。

In [4]:
data.cache
val numData = data.count
numData

7395

数据格式清理，把（“）去掉，还有？用0代替

In [2]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors
val data = records.map{
    r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed(r.size -1).toInt
    val features = trimmed.slice(4,r.size-1).map(d => if(d=="?") 0.0 else d.toDouble)
    LabeledPoint(label, Vectors.dense(features))
}

因为朴素贝叶斯模型要求特征值非负，所以需要为朴素贝叶斯模型单独构建一份输入特征向量的数据，将负特征值设为0

In [14]:
val nbData = records.map{
    r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed(r.size -1).toInt
    val features = trimmed.slice(4,r.size-1).map(d => if(d=="?") 0.0 else d.toDouble).map(d => if (d<0) 0.0 else d )
    LabeledPoint(label, Vectors.dense(features))
}

## 二、训练分类模型
为了比较不同模型的性能，我们将分别训练逻辑回归，SVM，朴素贝叶斯和决策树，然后对每个模型的结果进行比较。
### Kaggle/StumbleUpon evergreen 的分类数据集中训练分类模型
首先我们引入相应的依赖包，并且设置逻辑回归和SVM的迭代次数，为决策树设置最大树深度。  

In [52]:
import org.apache.spark.mllib.classification.LogisticRegressionWithSGD
import org.apache.spark.mllib.classification.SVMWithSGD
import org.apache.spark.mllib.classification.NaiveBayes
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.configuration.Algo
import org.apache.spark.mllib.tree.impurity.Entropy
val numIterations = 10
val maxTreeDepth = 5

首先 先训练逻辑回归模型：

In [9]:
val lrModel = LogisticRegressionWithSGD.train(data, numIterations)
lrModel

0.5

训练SVM模型：

In [10]:
val svmModel = SVMWithSGD.train(data, numIterations)
svmModel

0.0

训练朴素贝叶斯（使用处理过的非负特征值的数据）

In [15]:
val nbModel = NaiveBayes.train(nbData)
nbModel

org.apache.spark.mllib.classification.NaiveBayesModel@72b0e188

最后训练 决策树：

In [16]:
val dtModel = DecisionTree.train(data, Algo.Classification, Entropy, maxTreeDepth)
dtModel

DecisionTreeModel classifier of depth 5 with 61 nodes

## 三、使用分类模型
现在已经有四个训练好的模型，接下来我们中这些模型中进行预测。目前我们将使用同样的训练数据来解释每个模型的预测方法。
### 在Kaggle/StumbleUpon evergreen数据集上进行预测
以逻辑回归模型为例（其它模型处理方法类似）：

In [17]:
val dataPoint = data.first
val prediction = lrModel.predict(dataPoint.features)
prediction

1.0

可以看到对于训练模型中第一个样本，模型预测值为1，接下来我们验证一下这个样本真正的标签：

In [18]:
val trueLabel = dataPoint.label
trueLabel

0.0

可以看到预测结果错了！
我们可以将RDD［Vector］整体作为输入做预测：

In [19]:
val predictions = lrModel.predict(data.map(lp => lp.features))
predictions.take(5)

Array(1.0, 1.0, 1.0, 1.0, 1.0)

## 四、评估分类模型的性能
而分类中使用的评估方法包括：预测正确率和错误率，准确率和召回率  
### 预测正确率和错误率
在二分类中，预测正确率可能是最简单测评方式，正确率等于训练样本中被正确分类的数目除以总样本数，类似地，错误率等于样本中被错误分类的样本数目除以总样本数。   
我们通过对输入特征进行预测并将预测值和实际标签进行比较，计算出模型中训练数据上的正确率。将对正确分类的样本数目求和并除以样本总数，得到平均分类正确率：

In [20]:
val lrTotalCorrect = data.map{ point =>
    if ( lrModel.predict(point.features) == point.label) 1 else 0
}.sum
val lrAccuracy = lrTotalCorrect / data.count
lrAccuracy

0.5146720757268425

我们得到51.5%的正确率，结果看起来不是很好。我们的模型仅仅预测对了一半的训练模型，和随机预测差不多。

In [22]:
val svmTotalCorrect = data.map{point=>
    if(svmModel.predict(point.features) == point.label) 1 else 0
}.sum
val nbTotalCorrect = nbData.map{ point =>
    if(nbModel.predict(point.features) == point.label) 1 else 0
}.sum

注意，决策树的预测阀值需要明确给出

In [23]:
val dtTotalCorrect = data.map{ point =>
    val score = dtModel.predict(point.features)
    val predicted = if(score > 0.5) 1 else 0
    if( predicted == point.label ) 1 else 0
}.sum

svm模型正确率：

In [24]:
val svmAccuracy = svmTotalCorrect / numData
svmAccuracy

0.5146720757268425

朴素贝叶斯正确率：

In [25]:
val nbAccuracy = nbTotalCorrect / numData
nbAccuracy

0.5803921568627451

决策树的正确率：

In [26]:
val dtAccuracy = dtTotalCorrect / numData
dtAccuracy

0.6482758620689655

### 准确率和召回率
在信息检索中，准确率通常用于评价结果的质量，而召回率用来评价结果的完整性
#### 基本概念
True Positives,TP：预测为正样本，实际也为正样本的特征数  
False Positives,FP：预测为正样本，实际为负样本的特征数（错预测为正样本了，所以叫False）  
True Negatives,TN：预测为负样本，实际也为负样本的特征数  
False Negatives,FN：预测为负样本，实际为正样本的特征数（错预测为负样本了，所以叫False）  

TP+FP+FN+FN：特征总数(样本总数)  
TP+FN：实际正样本数  
FP+TN：实际负样本数  
TP+FP：预测结果为正样本的总数  
TN+FN：预测结果为负样本的总数
#### ROC曲线和AUC
##### True Positive Rate(TPR)和False Positive Rate(FPR)分别构成ROC曲线的y轴和x轴。  
TPR=TP/(TP+FN)，实际正样本中被预测正确的概率  
FPR=FP/(FP+TN)，实际负样本中被错误预测为正样本的概率   
其中曲线下面的面积就是AUC，面积越大模型就越好
<img src="./images/1.jpeg" width = "400px"/>
##### Precision和Recall（有人中文翻译成召回率）则分别构成了PR曲线的y轴和x轴.  
Precision=TP/(TP+FP)，预测结果为有多少正样本是预测正确了的  
Recall=TP/(TP+FN)，召回率很有意思，这个其实就=TPR，相对于Precision只不过参考样本从预测总正样本数结果变成了实际总正样本数
<img src="./images/2.jpeg" width = "400px"/>
一般来说，上面的比下面的好（绿线比红线好），但是有时候模型没有单纯的谁比谁好（比如图二的蓝线和青线），那么选择模型还是要结合具体的使用场景。  
下面是两个场景：
1. 地震的预测
对于地震的预测，我们希望的是RECALL非常高，也就是说每次地震我们都希望预测出来。这个时候我们可以牺牲PRECISION。情愿发出1000次警报，把10次地震都预测正确了；也不要预测100次对了8次漏了两次。
2. 嫌疑人定罪
基于不错怪一个好人的原则，对于嫌疑人的定罪我们希望是非常准确的。及时有时候放过了一些罪犯（recall低），但也是值得的。


> 因为PR曲线下的面积和ROC曲线下的面积经过归一化（最小值为0，最大值为1），我们可以用这些度量方法比较不同参数配置下的模型，甚至可以比较完全不同的模型。这两个方法中模型评估和选择上也很常用

Mllib内置了一系列方法用来计算二分类的PR和ROC曲线下的面积。下面我们针对每一个模型来计算这些指标：

In [27]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
val metrics = Seq(lrModel,svmModel).map{ model =>
    val scoreAndLabels = data.map{ point =>
        (model.predict(point.features),point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName,metrics.areaUnderPR,metrics.areaUnderROC)
}

我们之前已经训练朴素贝叶斯模型并计算准确率，其中使用的数据集是nbData版本，这里用同样的数据集计算分类的结果。

In [29]:
val nbMetrics = Seq(nbModel).map{ model =>
    val scoreAndLabels = nbData.map{ point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName, metrics.areaUnderPR,metrics.areaUnderROC)
}

因为DecisionTreeModel模型么有实现其它三个模型都有的ClassificationModel接口，因此我们需要单独为这个模型编写如下代码：

In [30]:
val dtMetrics = Seq(dtModel).map{ model =>
    val scoreAndLabels = data.map{ point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (model.getClass.getSimpleName,metrics.areaUnderPR,metrics.areaUnderROC)
}

In [31]:
val allMetrics = metrics ++ nbMetrics ++ dtMetrics
allMetrics.foreach{ case (m, pr, roc) =>
    println(f"$m, Area under PR: ${pr * 100.0}%2.4f%%, Area under ROC: ${roc * 100.0}%2.4f%%")
}

LogisticRegressionModel, Area under PR: 75.6759%, Area under ROC: 50.1418%
SVMModel, Area under PR: 75.6759%, Area under ROC: 50.1418%
NaiveBayesModel, Area under PR: 68.0851%, Area under ROC: 58.3559%
DecisionTreeModel, Area under PR: 74.3081%, Area under ROC: 64.8837%


我们可以看到所有的模型得到的平均准确率差不多。  
逻辑回归和SVM的AUC的结果中0.5左右，表明这两个模型并不比随机好。朴素贝叶斯模型和决策树模型性能稍微好些，AUC分别是0.58和0.65.但是，在二分类问题上这个性能并不是非常好。

## 五、改进模型性能以及参数调优

### 特征标准化
我们使用的许多模型对输入数据的分布和规模有一些固有的假设，其中最常见的假设形式是特征满足正态分布。下面让我们进一步研究特征是如何分布的。  
具体做法，我们先将特征向量用RowMatrix类表示成MLlib中的分布矩阵。RowMatrix是一个由向量组成的RDD，其中每个向量是分布矩阵的一行。   
RowMatrix类中有一些方便操作矩阵的方法，其中一个方法可以计算矩阵每列的统计特性：  
输出矩阵每列的均值：

In [32]:
 import org.apache.spark.mllib.linalg.distributed.RowMatrix
 val vectors = data.map( lp => lp.features )
 val matrix = new RowMatrix(vectors)
 val matrixSummary = matrix.computeColumnSummaryStatistics()
 println(matrixSummary.mean)

[0.4122580529952672,2.761823191986608,0.4682304732861389,0.21407992638350232,0.09206236071899916,0.04926216043908053,2.255103452212041,-0.10375042752143335,0.0,0.05642274498417851,0.02123056118999324,0.23377817665490194,0.2757090373659236,0.615551048005409,0.6603110209601082,30.07707910750513,0.03975659229208925,5716.598242055447,178.75456389452353,4.960649087221096,0.17286405047031742,0.10122079189276552]


In [33]:
println(matrixSummary.min)

[0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.045564223,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0]


In [34]:
println(matrixSummary.max)

[0.999426,363.0,1.0,1.0,0.980392157,0.980392157,21.0,0.25,0.0,0.444444444,1.0,0.716883117,113.3333333,1.0,1.0,100.0,1.0,207952.0,4997.0,22.0,1.0,1.0]


In [46]:
println(matrixSummary.variance)

[0.10974244167559001,74.30082476809639,0.04126316989120241,0.02153343633200108,0.009211817450882448,0.005274933469767946,32.53918714591821,0.09396988697611545,0.0,0.0017177410346628928,0.020782634824610638,0.0027548394224293036,3.683788919674426,0.2366799607085986,0.22433071201674218,415.8785589543846,0.03818116876739597,7.877330081138463E7,32208.116247426184,10.45300904576431,0.03359363403832393,0.006277532884214705]


输出每列中非零项的数目：

In [35]:
println(matrixSummary.numNonzeros)

[5053.0,7354.0,7172.0,6821.0,6160.0,5128.0,7350.0,1257.0,0.0,7362.0,157.0,7395.0,7355.0,4552.0,4883.0,7347.0,294.0,7378.0,7395.0,6782.0,6868.0,7235.0]


使用StandardScaler来完成数据的标准化（x-u）/sqrt(variance)

In [43]:
import org.apache.spark.mllib.feature.StandardScaler
val scaler = new StandardScaler(withMean = true, withStd = true).fit(vectors)
val scaledData = data.map(lp => LabeledPoint(lp.label,scaler.transform(lp.features)))

[0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]


观察第一行原始数据

In [45]:
println( data.first.features )

[0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]


观察标准化后的第一行原始数据

In [44]:
println( scaledData.first.features )

[1.137647336497678,-0.08193557169294771,1.0251398128933331,-0.05586356442541689,-0.4688932531289357,-0.3543053263079386,-0.3175352172363148,0.3384507982396541,0.0,0.828822173315322,-0.14726894334628504,0.22963982357813484,-0.14162596909880876,0.7902380499177364,0.7171947294529865,-0.29799681649642257,-0.2034625779299476,-0.03296720969690391,-0.04878112975579913,0.9400699751165439,-0.10869848852526258,-0.2788207823137022]


可以看出，第一个特征已经应用标准差公式被转换来。为确认这一点，可以让第一个特征减去均值，然后除以标准差：

In [49]:
println( ( 0.789131 - 0.4122580529952672 ) / math.sqrt( 0.10974244167559001 ) )

1.137647336497678


现在我们使用标准化的数据重训练模型。这里只训练逻辑回归（因为决策树和朴素贝叶斯不受特征标准化的影响），并说明特征标准化的影响：

In [55]:
val lrModelScaled = LogisticRegressionWithSGD.train( scaledData, numIterations )
val lrTotalCorrectScaled = scaledData.map { point =>
    if (lrModelScaled.predict(point.features) == point.label) 1 else 0
}.sum
val lrAccuracyScaled = lrTotalCorrectScaled / numData
val lrPredictionsVsTrue = scaledData.map{ point =>
    (lrModelScaled.predict(point.features),point.label)
}
val lrMetricsScaled = new BinaryClassificationMetrics(lrPredictionsVsTrue)
val lrPr = lrMetricsScaled.areaUnderPR
val lrRoc = lrMetricsScaled.areaUnderROC
println(f"${lrModelScaled.getClass.getSimpleName}\nAccuracy:${lrAccuracyScaled * 100}%2.4f%%\nArea under PR: ${lrPr * 100.0}%2.4f%%\nArea under ROC: ${lrRoc * 100.0}%2.4f%%")

LogisticRegressionModel
Accuracy:62.0419%
Area under PR: 72.7254%
Area under ROC: 61.9663%


从结果可以看出，通过简单对特征标准化，就提高了逻辑回归的准确率，并将AUC从随机50%提升到62%。

### 其它特征

我们已经看到，需要注意对特征进行标准化和归一化，这对模型性能可能有重要的影响。在这个示列中，我们仅仅使用了部分特征，却完全忽略了类别（category）变量和样板（boilerplate）列的文本内容。  
这样做是为了便于介绍。现在我们再来评估一下添加其它特征，比如类别特征对性能的影响。  
首先，我们查看所有的类别，并对每个类别做一个索引对映射，这里索引可以用于类别特征做1-of-k编码。  
不同类别的输出：

In [56]:
val categories = records.map(r => r(3)).distinct.collect.zipWithIndex.toMap
val numCategories = categories.size
println(categories)

Map("weather" -> 0, "sports" -> 6, "unknown" -> 4, "computer_internet" -> 12, "?" -> 11, "culture_politics" -> 3, "religion" -> 8, "recreation" -> 2, "arts_entertainment" -> 9, "health" -> 5, "law_crime" -> 10, "gaming" -> 13, "business" -> 1, "science_technology" -> 7)


下面代码会计算出类别的数目：

In [57]:
println(numCategories)

14


因此，我们需要创建一个长为14的向量来表示类别特征，然后根据每个样本所属类别索引，对相应的维度赋值为1，其它为0，我们假定这个新的特征向量和其它的数值特征向量一样：  
我们可以看到如下的输出，其中第一部分是一个14维的向量，向量中类别对应索引那一维为1.

In [58]:
val dataCategories = records.map{ r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed(r.size -1).toInt
    val categoryIdx = categories(r(3))
    val categoryFeatures = Array.ofDim[Double](numCategories)
    categoryFeatures(categoryIdx) = 1.0
    val otherFeatures = trimmed.slice(4,r.size -1).map( d => if(d == "?") 0.0 else d.toDouble)
    val features = categoryFeatures ++ otherFeatures
    LabeledPoint( label, Vectors.dense(features))
}
println( dataCategories.first )

(0.0,[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])


同样，因为我们的原始数据没有标准化，所以中训练这个扩展数据集之前，应该使用同样的StandardScaler方法对其进行标准化转换：

In [60]:
val scalerCats = new StandardScaler(withMean = true, withStd = true).fit(dataCategories.map(lp => lp.features))
val scaledDataCats = dataCategories.map(lp => LabeledPoint(lp.label,scalerCats.transform(lp.features)))

可以使用如下代码看到标准化之前的特征：

In [61]:
println( dataCategories.first.features )

[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]


标准化后的特征：

In [62]:
println( scaledDataCats.first.features )

[-0.02326210589837061,2.7207366564548514,-0.4464212047941535,-0.22052688457880879,-0.028494000387023734,-0.2709990696925828,-0.23272797709480803,-0.2016540523193296,-0.09914991930875496,-0.38181322324318134,-0.06487757239262681,-0.6807527904251456,-0.20418221057887365,-0.10189469097220732,1.137647336497678,-0.08193557169294771,1.0251398128933331,-0.05586356442541689,-0.4688932531289357,-0.3543053263079386,-0.3175352172363148,0.3384507982396541,0.0,0.828822173315322,-0.14726894334628504,0.22963982357813484,-0.14162596909880876,0.7902380499177364,0.7171947294529865,-0.29799681649642257,-0.2034625779299476,-0.03296720969690391,-0.04878112975579913,0.9400699751165439,-0.10869848852526258,-0.2788207823137022]


现在，可以用扩展后的特征来训练新的逻辑回归模型了，然后我们再评估其性能：

In [64]:
val lrModelScaledCats = LogisticRegressionWithSGD.train( scaledDataCats, numIterations )
val lrTotalCorrectScaledCats = scaledDataCats.map { point =>
    if (lrModelScaledCats.predict(point.features) == point.label) 1 else 0
}.sum
val lrAccuracyScaledCats = lrTotalCorrectScaledCats / numData
val lrPredictionsVsTrueCats = scaledDataCats.map{ point =>
    (lrModelScaledCats.predict(point.features),point.label)
}
val lrMetricsScaledCats = new BinaryClassificationMetrics(lrPredictionsVsTrueCats)
val lrPrCats = lrMetricsScaledCats.areaUnderPR
val lrRocCats = lrMetricsScaledCats.areaUnderROC
println(f"${lrModelScaledCats.getClass.getSimpleName}\nAccuracy:${lrAccuracyScaledCats * 100}%2.4f%%\nArea under PR: ${lrPrCats * 100.0}%2.4f%%\nArea under ROC: ${lrRocCats * 100.0}%2.4f%%")

LogisticRegressionModel
Accuracy:66.5720%
Area under PR: 75.7964%
Area under ROC: 66.5483%


## 使用正确的数据格式
模型性能的另一个关键部分是对每一个模型使用正确的数据格式。前面对数值向量应用朴素贝叶斯模型得到了非常差的结果？  
在这里，我们知道MLlib实现了多项式模型，并且该模型可以处理计数形式的数据。这包括二元表示的类型特征（比如前面提到的1-of-k表示）或者频率数据（比如一个文档中单词出现的频率）。我开始时使用的数值特征并不符合假定的输入分布，所以模型性能不好也并不是意料之外。  
为了更好的说明，我们仅仅使用类型特征，而1-of-k编码的类型特征更符合朴素贝叶斯模型，我们用如下代码构建数据集：

In [65]:
val dataNB = records.map{ r =>
    val trimmed = r.map(_.replaceAll("\"",""))
    val label = trimmed( r.size - 1 ).toInt
    val categoryIdx = categories( r(3) )
    val categoryFeatures = Array.ofDim[Double](numCategories)
    categoryFeatures(categoryIdx) = 1.0
    LabeledPoint( label, Vectors.dense(categoryFeatures))
}

接下来，我们重新训练朴素贝叶斯模型对它的性能进行评估：

In [66]:
val nbModelCats = NaiveBayes.train(dataNB)
val nbTotalCorrectCats = dataNB.map{ point =>
    if( nbModelCats.predict(point.features) == point.label) 1 else 0
}.sum
val nbAccuracyCats = nbTotalCorrectCats / numData
val nbPredictionsVsTrueCats = dataNB.map{ point =>
    (nbModelCats.predict(point.features), point.label)
}
val nbMetricsCats = new BinaryClassificationMetrics(nbPredictionsVsTrueCats)
val nbPrCats = nbMetricsCats.areaUnderPR
val nbRocCats = nbMetricsCats.areaUnderROC
println(f"${nbModelCats.getClass.getSimpleName}\nAccuracy:${nbAccuracyCats * 100}%2.4f%%\nArea under PR: ${nbPrCats * 100.0}%2.4f%%\nArea under ROC: ${nbRocCats * 100.0}%2.4f%%")

NaiveBayesModel
Accuracy:60.9601%
Area under PR: 74.0522%
Area under ROC: 60.5138%


可见，使用格式正确的输入数据后，朴素贝叶斯的性能从58%提高到了60%

### 模型参数调优
前面展示了模型性能的影响因素：特征提取，特征选择，输入数据的格式和模型对数据分布的假设。但是到目前为止，我们对模型参数对讨论只是一笔带过，而实际上对于模型性能影响很大。  
#### 1 线性模型
Mllib默认对train方法对每个模型的参数都是你用默认值。接下来让我们深入了解一下这些参数。

In [69]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.optimization.Updater
import org.apache.spark.mllib.optimization.SimpleUpdater
import org.apache.spark.mllib.optimization.L1Updater
import org.apache.spark.mllib.optimization.SquaredL2Updater
import org.apache.spark.mllib.classification.ClassificationModel

定义辅助函数，根据给定输入训练模型：

In [70]:
def trainWithParams( input: RDD[LabeledPoint], regParam: Double, numIterations: Int, updater: Updater, stepSize: Double) = {
    val lr = new LogisticRegressionWithSGD
    lr.optimizer.setNumIterations(numIterations).setUpdater(updater).setRegParam(regParam).setStepSize(stepSize)
    lr.run(input)
}

最后，我们定义第二个辅助函数并根据输入数据和分类模型，计算相关的AUC：

In [71]:
def createMetrics(label: String, data: RDD[LabeledPoint],model:ClassificationModel) = {
    val scoreAndLabels = data.map{ point =>
        (model.predict(point.features), point.label)
    }
    val metrics = new BinaryClassificationMetrics( scoreAndLabels )
    ( label, metrics.areaUnderROC)
}

In [72]:
scaledDataCats.cache

MapPartitionsRDD[342] at map at <console>:64

（1） 迭代  
大多数机器学习的方法需要迭代训练，并且经过一定次数的迭代之后收敛到某个解（既最小损失函数时的最优权重向量）。SGD收敛到合适的解需要迭代次数相对较少，但是要进一步提升性能则需要更多次迭代，为方便解释，这里设置不同的迭代次数numIterations，然后比较AUC的结果：

In [73]:
val iterResults = Seq(1 ,5 ,10, 50).map { param =>
    val model = trainWithParams(scaledDataCats, 0.0, param, new SimpleUpdater, 1.0)
    createMetrics(s"$param iterations", scaledDataCats, model)
}
iterResults.foreach{ case (param, auc) => println(f"$param, AUC = ${auc * 100}%2.2f%%") }

1 iterations, AUC = 64.95%
5 iterations, AUC = 66.62%
10 iterations, AUC = 66.55%
50 iterations, AUC = 66.81%


（2）步长  
在SGD中，在训练每个样本并更新的权重向量时，步长用来控制算法在最徒的梯度方向上该前进多远。较大的步长收敛较快，但是步长太大可能导致收敛到局部最优解。  
下面的结果可以看出步长增大过大对性能有负面的影响：

In [76]:
val stepResults = Seq(0.001, 0.01, 0.1, 10.0).map{ param =>
    val model = trainWithParams(scaledDataCats, 0.0, numIterations, new SimpleUpdater, param)
    createMetrics(s"$param step size", scaledDataCats, model)
}
stepResults.foreach{ case ( param, auc ) => println( f"$param, AUC = ${auc * 100}%2.2f%%")}

0.001 step size, AUC = 64.97%
0.01 step size, AUC = 64.96%
0.1 step size, AUC = 65.52%
10.0 step size, AUC = 61.92%


（3）正则化  
前面逻辑回归的代码中简单提及了Updater类，该类中MLlib中实现了正则化。正则化通过限制模型的复杂度避免模型中训练数据中过度拟合。  
MLlib中可用的正则化形式有如下几个。
SimpleUpdater:相当于没有正则化，是逻辑回归的默认设置  
SquaredL2Updater: 这个正则项基于权重向量的L2正则化，是SVM模型的默认值。  
L1Updater:这个正则项基于权重向量的L1正则化，会导致得到一个稀疏的权重向量（不重要的权重的值接近0）  
下面结果可以看出：低等级的正则化对模型的影响不大。然而，增大正则化可以看到欠拟合会导致模型性能降低。

In [79]:
val regResults = Seq(0.001, 0.01, 0.1, 10.0).map{ param =>
    val model = trainWithParams(scaledDataCats, param, numIterations,new SquaredL2Updater,1.0)
    createMetrics(s"$param L2 regularization parameter", scaledDataCats, model)
}
regResults.foreach{ case (param, auc) => println(f"$param, AUC=${auc * 100}%2.2f%%") }

0.001 L2 regularization parameter, AUC=66.55%
0.01 L2 regularization parameter, AUC=66.55%
0.1 L2 regularization parameter, AUC=66.63%
10.0 L2 regularization parameter, AUC=35.33%


### 决策树
决策树模型在一开始使用原始数据训练时获得了最好的性能。当时设置了参数maxDepth用来控制决策树的最大深度，进而控制模型的复杂度。而树的深度越大，得到的模型越复杂，但有能力更好地拟合数据。  
对于分裂问题，我们需要为决策树模型选择以下两种不纯度度量方式：Gini或者Entropy。  
* 树的深度和不纯度调优
下面我们来说明树的深度对模型性能的影响，其中使用与评估逻辑回归模型类似的评估方法（AUC）  
首先创建一个辅助函数：

In [80]:
import org.apache.spark.mllib.tree.impurity.Impurity
import org.apache.spark.mllib.tree.impurity.Entropy
import org.apache.spark.mllib.tree.impurity.Gini

def trainDTWithParams(input: RDD[LabeledPoint], maxDepth: Int, impurity: Impurity) = {
    DecisionTree.train(input, Algo.Classification, impurity, maxDepth)
}

接着，准备计算不同树深度配置下的AUC。因为不需要对数据进行标准化，所以我们将使用样例中原始的数据。  
首先，通过使用Entropy不纯度并改变树的深度训练模型：  
计算结果如下：

In [81]:
val dtResultsEntropy = Seq(1,2,3,4,5,10,20).map{ param =>
    val model = trainDTWithParams(data, param,Entropy)
    val scoreAndLabels = data.map{ point =>
        val score = model.predict(point.features)
        (if (score > 0.5) 1.0 else 0.0, point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (s"$param tree depth", metrics.areaUnderROC)
}
dtResultsEntropy.foreach{case (param,auc) => println(f"$param, AUC = ${auc * 100}%2.2f%%")}

1 tree depth, AUC = 59.33%
2 tree depth, AUC = 61.68%
3 tree depth, AUC = 62.61%
4 tree depth, AUC = 63.63%
5 tree depth, AUC = 64.88%
10 tree depth, AUC = 76.26%
20 tree depth, AUC = 98.45%


接下来，我们采用Gini不纯度进行类似的计算，计算结果如下：  

1 tree depth, AUC = 59.33%  
2 tree depth, AUC = 61.68%  
3 tree depth, AUC = 62.61%  
4 tree depth, AUC = 63.63%  
5 tree depth, AUC = 64.89%  
10 tree depth, AUC = 78.37%  
20 tree depth, AUC = 98.87%  
从结果中可以看出，提高树的深度可以得到更精确的模型，然而树的深度越大，模型对训练数据过拟合程度越严重。（两种不同的纯度方法对性能的影响较小）

### 朴素贝叶斯
最后，让我们看看lamda参数对朴素贝叶斯模型的影响。该模型可以控制相加式平滑（additive smoothing）,解决数据中某个类别和某个特征值的组合没有同时出现的问题。  
首先需要创建一个方便调用的辅助函数，用来训练不同lamba级别下的模型：  
结果如下：

In [83]:
def trainNBWithParams(input: RDD[LabeledPoint],lambda: Double) = {
    val nb = new NaiveBayes
    nb.setLambda(lambda)
    nb.run(input)
}
val nbResults = Seq(0.001,0.01,0.1,1.0,10.0).map{ param =>
    val model = trainNBWithParams(dataNB, param)
    val scoreAndLabels = dataNB.map{ point =>
        (model.predict(point.features), point.label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    (s"$param lambda", metrics.areaUnderROC)
}
nbResults.foreach{ case(param, auc) => println(f"$param, AUC=${auc * 100}%2.2f%%")}

0.001 lambda, AUC=60.51%
0.01 lambda, AUC=60.51%
0.1 lambda, AUC=60.51%
1.0 lambda, AUC=60.51%
10.0 lambda, AUC=60.51%


### 交叉验证
交叉验证让我们使用一部分数据训练模型，将另一部分用来评估模型性能。如果模型在训练以外的新数据进行了测试，我们便可以由此评估模型对新数据的泛化能力。  
首先，我们将数据集分成60%的训练集和40%的测试集（为了方便解释，我们在代码中使用一个固定的随机种子123来保证每次实验能得到相同的结果）：

In [84]:
val trainTestSplit = scaledDataCats.randomSplit(Array(0.6, 0.4), 123)
val train = trainTestSplit(0)
val test = trainTestSplit(1)

接下来在不同的正则化参数下评估模型的性能（这里依然使用AUC）。注意我们在正则化参数之间设置了很小的步长，为的是更好解释AUC在各个正则化参数下的变化，同时这个例子的AUC的变化也很小：  
计算测试集的模型性能，具体结果如下：

In [86]:
val regResultsTest = Seq(0.0, 0.001, 0.0025, 0.005, 0.01).map{ param =>
    val model = trainWithParams( train, param, numIterations, new SquaredL2Updater, 1.0)
    createMetrics(s"$param L2 regularization parameter", test, model)
}
regResultsTest.foreach{ case(param, auc) => println(f"$param, AUC = ${auc * 100}%2.6f%%")}

0.0 L2 regularization parameter, AUC = 67.173110%
0.001 L2 regularization parameter, AUC = 67.173110%
0.0025 L2 regularization parameter, AUC = 67.173110%
0.005 L2 regularization parameter, AUC = 67.140600%
0.01 L2 regularization parameter, AUC = 67.175986%


接着让我们比较一下在训练模型性能：  
0.0 L2 regularization parameter, AUC = 66.260311%  
0.001 L2 regularization parameter, AUC = 66.260311%  
0.0025 L2 regularization parameter, AUC = 66.2603110%  
0.005 L2 regularization parameter, AUC = 66.238294%  
0.01 L2 regularization parameter, AUC = 66.238294%  
从上面的结果可以看出，当我们的训练集和测试集相同时，通常在正则化参数比较小点情况下可以得到最高的性能。这是因为我们的模型在较低的正则化下学习了所有的数据。既过拟合的情况下达到更高的性能。  
相反，当训练集和测试集不同时，通常较高正则化可以得到较高的测试性能。
